In [5]:
from torchvision import models
import torch
import torch.nn as nn
from torch.profiler import profile, ProfilerActivity
import torch.distributed as dist
import torch.multiprocessing as mp
from torchsummary import summary
from deepspeed.profiling.flops_profiler import get_model_profile

In [2]:
num_parameters = sum(p.numel() for p in models.resnet18().parameters())
num_parameters

11689512

In [4]:
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
with torch.cuda.device(0):
    model = models.vgg16()
    batch_size = 1
    flops, macs, params = get_model_profile(model=model, # model
                                     input_shape=(batch_size, 3, 224, 224), # input shape or input to the input_constructor
                                     print_profile=True, # prints the model graph with the measured profile attached to each module
                                     top_modules=3, # the number of top modules to print aggregated profile
                                     warm_up=3, # the number of warm-ups before measuring the time of each module
    )
    print("{:<30}  {:<8}".format("Batch size: ", batch_size))
    print('{:<30}  {:<8}'.format('Number of MACs: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))
    print('{:<30}  {:<8}'.format('Number of FLOPs: ', flops))


-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 3:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per gpu:                                               138.36 M
params of model = params per GPU * mp_size:                   1       
fwd MACs per GPU:                                             15.47 GMACs
fwd flops per GPU:                                            30.97 G 
fwd flops of model = fwd flops per GPU * mp_size:             1       
fwd latency:                                                  59.17 ms
fwd FLOPS per GPU = fwd flops per GPU / fwd latency:    

In [3]:
import pandas as pd
pd.options.display.expand_frame_repr = False

df = pd.read_csv("bench/results/20220511-011500/result.csv", sep="\t")
df.pivot_table(index=["op", "nchannels"], columns="nthreads", values=["comm_time", "comp_time"], aggfunc="median")

Empty DataFrame
Columns: []
Index: [(conv, 0), (conv, 1), (conv, 2), (conv, 3), (conv, 4), (conv, 5), (conv, 6), (conv, 7), (conv, 8)]

nthreads       0        64       128     256     512
op  nchannels                                       
nop 0          0.0      NaN      NaN     NaN     NaN
    1          NaN  23.8585  16.6105  9.1990  7.3385
    2          NaN  12.6755   9.1275  4.9775  3.9605
    3          NaN   8.5915   6.2380  3.7210  3.0330
    4          NaN   6.6895   4.8350  3.1390  2.7975
    5          NaN   5.4815   4.0735  2.8845  2.6580
    6          NaN   4.6605   3.6675  2.7785  2.6665
    7          NaN   4.1445   3.2525  2.8095  2.6015
    8          NaN   3.7545   3.0675  2.7585  2.5390